## Jetson Nano 视觉识别
### 在Jetson Nano上使用ssd_mobilenet_v2_coco视觉模型实现对环境中物体的识别。


 <span style="color: brown;"> 程序运行准备：
 <br>
  本程序要在Jetson Nano环境中执行。为能让JetBot小车自由运动，Jetson Nano系统主板不能连接电源线、鼠标、显示器及其他USB连线。必须通过网络登录后进入网页版Jupyter lab执行。Jetson Nano的网址在Jetson Nano系统连接到本地网络时，显示在Jetson Nano系统的液晶显示屏上，形式为：192.168.1.XXX(XXX为两位或三位数字)。<br>
  程序启动部步骤：<br>
  1.装上烧入了JetPack软件镜像的SD卡。<br>
  2.在Jetson Nano的HDMI接口连接显示器、在USB接口中分别接入键盘和鼠标，启动jetson nano系统。<br>
  3.进入Ubuntu18.4,在此界面里设定Jetson Nano系统本地网络的连接。如图-1。<br>
  4.拔除显示器、键盘、鼠标、USB和充电电源线，重新启动Jetson Nano，这时可以在智能小车的液晶显示板上看见Jetson Nano的本地网址。如图-2。<br>
  5.在接入料本地网络的外部计算机的浏览器上输入Jetson Nano的本地网址，即可连接到Jetson Nano系统。如图-3。<br>
<span>

<span style="color: brown;">
本文对程序的说明使用了两种颜色，黑色字体表示基本说明，初学者应该掌握的知识，绿色字体是详细说明，介绍的程序运行的细节，需要一些背景知识，用于深入学习参考。<br>
<span>



<img src="../image/Ubuntu1.jpg" width="300" height="200">
<img src="../image/./液晶1.jpg" width="300" height="200">
<img src="../image/./远程1.jpg" width="300" height="200">


使用现存功能强大、应用可靠多物体视觉模型，是实现计算机视觉功能简易快速的方法。<br>
这种功能强大应用可靠多物体视觉模型称为预训练模型(pretrained model),这些模型是在大量数据上预先训练好的，可以被用来解决相似或相关领域的问题，而不需要从头开始训练。预训练模型可以显著减少模型在新任务上的训练时间和所需的训练数据量，提高模型的泛化能力和准确性。在自然语言处理（NLP）、计算机视觉（CV）等领域，预训练模型已经成为了标准做法之一。<br>
所以，本专题着重介绍在Jetson Nano上使用现存的多物体视觉模型进行周边环境中各种物体的识别的算法和程序实现，让AI初学者理解一般意义上的视觉模型工作原理以及在Jetson Nano上视觉推理的算法实现。对于在VAIC竞赛中为提高特定目标识别性能，对预训练模型微调，在以下的专题中再进行详细介绍。<br>
注：程序中使用的Jetson Nano系统、摄像头及其运载Jetson Nano的小车的介绍，请参考本目录的"Jetson-Nano系统简介.ipynb"。<br>

In [3]:
from jetbot import ObjectDetector

model = ObjectDetector('./model/ssd_mobilenet_v2_coco.engine')


from jetbot import ObjectDetector 从NVIDIA jetbot函数库中导入ObjectDetector模块。<br>
jetbot 是一个为 NVIDIA Jetson Nano 开发的 Python 库，旨在简化机器人项目（如自动驾驶小车、监控机器人等）的开发。这个库提供了一系列工具和模块，帮助开发者快速搭建和部署基于 Jetson Nano 的项目。<br>
在 jetbot 库中，ObjectDetector 是一个特别重要的组件，它用于检测图像中的对象。<br>

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')引进ssd_mobilenet_v2_coco视觉模型。<br>
ssd_mobilenet_v2_coco是一个常用于目标检测任务的深度学习模型。是由前期的MobilNet v2模型和后期的SSD模型组合而成，并在COCO数据集训练，因而得名。<br>
<span style="color: green;">
简单解释如下：<br>
1.SSD(Single Shot MultiBox Detector):<br>
  Single Shot（单次检测）该模型在一次前向传播中完成目标检测任务，因此速度非常快。<br>
  MultiBox：这是SSD中指代多个候选边界框的生成和处理。<br>
  Detector（检测器）：SSD的主要功能是进行目标检测，即识别图像中的目标物体，并将它们分类到预定义的类别中，同时输出它们的位置（边界框）。<br>
2.MobileNet V2：<br>
  MobileNet V2是一个轻量级、高效的卷积神经网络（CNN）架构，专为移动设备和嵌入式视觉应用设计。它采用深度可分离卷积（Depthwise Separable Convolutions）技术，相比标准卷积大大减少了参数数量和计算量，非常适合在资源受限的环境中使用。<br>
3.COCO数据集：
  COCO（Common Objects in Context）是一个大型图像数据集，包含了80个对象类别的数十万张图片，被广泛用于目标检测、语义分割和关键点检测等任务。<br>
  SSD MobileNet V2 COCO 模型通常是在这个数据集上进行训练的，因此能够检测到诸如人、车、动物等常见物体。<br>
关于ssd_mobilenet_v2_coco模型网络结构和运行原理的详细说明在附文”SSD_MobileNet_V2_COCO模型技术说明.ipynb“中，有兴趣的读者可自行参考。<br>
从.engine后缀可以看出是此模型是经过TensorTR优化，能在NVIDIA Jetson系列产品的GPU上进行高速推理。<br>
<span>

In [5]:
def parse_data(file_path):
    items = []
    with open(file_path, 'r', encoding='utf-8') as file:
        item = {}
        for line in file:
            line = line.strip()
            if not line or line == "item {":  # Skip empty lines and start of a new item
                continue
            if line == "}":  # End of current item
                items.append(item)
                item = {}
            else:
                try:
                    key, value = line.split(": ", 1)  # Only split on the first colon
                    key = key.strip()
                    value = value.strip().strip('"')
                    if key == "name" and key in item:  # Handle duplicate 'name' keys
                        item["zh_name"] = value
                    else:
                        item[key] = value
                except ValueError as e:  # Catch lines that don't have a colon
                    print(f"Error processing line: {line} - {e}")
    return items



file_path = './model/识别物体.json'
data = parse_data(file_path)
key='zh_name'
id=88
print(data[id]['id'],data[88][key])

88 毛毛熊


读出ssd_mobilenet_v2_coco识别COCO数据集包含的80个被识别物体类别标签及显示名，数据文件是本目录的"./model/识别物体.json"。

In [6]:
from jetbot import Camera
w=300
h=300
camera = Camera.instance(width=w, height=h)

导入jetbot的python库摄像机函数模块，设定图像的长宽并创建Camera实例。

In [7]:
detections = model(camera.value)

#print(detections)

将来自摄像机（camera）的捕获的图像（camera.value）传递给模型（model）进行识别，并将检测到的对象或特征存储在变量detections中。<br>
 <span style="color: green;">
detections格式如下：<br>
key="detection_boxes": list(int) #物体的边界框<br>
key="detection_classes"：list(int) # 类别索引列表<br>
key="detection_scores":list(float) # 置信度分数列表<br>
key="num_detections": int # 检测到的物体数量<br>
例如：<br>
{<br>
  "detection_boxes": [<br>
    [0.1, 0.2, 0.5, 0.6],  // 第一个物体的边界框 (xmin, ymin, xmax, ymax)<br>
    [0.4, 0.4, 0.7, 0.8],  // 第二个物体的边界框<br>
    [0.3, 0.3, 0.6, 0.7]   // 第三个物体的边界框<br>
  ],<br>
  "detection_classes": [1, 3, 17],  // 检测到的类别索引（例如1代表人，3代表汽车，17代表狗）<br>
  "detection_scores": [0.95, 0.89, 0.78],  // 每个检测的置信度分数<br>
  "num_detections": 3  // 检测到的物体数量<br>
}<br>
<span>

In [8]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

#display(detections_widget)

引入Jupyter Notebook 或其他交互式 Python 环境中用于创建和显示交互式的小部件（widget）。
1. `from IPython.display import display`:
  从 `IPython.display` 模块中导入 `display` 函数。`display` 函数用于在 Jupyter Notebook 中渲染或显示小部件或其他输出内容。

2. `import ipywidgets.widgets as widgets`:
  从 `ipywidgets` 库中导入 `widgets` 模块，`ipywidgets` 是一个用于在 Jupyter Notebook 中创建交互式 HTML 小部件的包。

3. `detections_widget = widgets.Textarea()`:
  创建了一个 `Textarea` 小部件，并将其赋值给变量 `detections_widget`。
  `Textarea` 是一个多行文本框，可以让用户输入或显示文本。在这个代码中，它被用来显示 `detections` 变量的内容。

4. `detections_widget.value = str(detections)`:
  将 `Textarea` 小部件的内容设置为 `detections` 变量的字符串形式。
  通过 `str(detections)`，将 `detections` 转换为字符串，然后将其作为文本显示在 `Textarea` 中。


In [9]:
image_number = 0
object_number = 0

#print(detections[image_number][object_number])

初始化image_number、object_number

In [10]:
from jetbot import Robot

robot = Robot()

from jetbot import Robot:
从jetbot中导入Robot类。jetbot包是专门为JetBot机器人平台设计的，包含了一系列用于控制机器人、处理摄像头输入、进行图像识别等功能的类和函数。

robot = Robot():
创建了一个Robot类的实例，并将其赋值给变量robot。通过这个实例，你可以访问Robot类中定义的所有方法和属性，从而控制JetBot机器人的行为。
Robot()的构造函数负责初始化机器人所需的硬件接口（如电机、摄像头等），并设置一些基本的配置。

In [11]:
from jetbot import bgr8_to_jpeg
import cv2
import numpy as np
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc,20.0,(300,300))
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
text_color = (255,255,255)
thickness = 2
#blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=w, height=h)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')
display(widgets.VBox([
    widgets.HBox([image_widget]),#, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)


定义了ipywidgets的小部件：<br>
image_widget 图像显示<br>
label_widget 标签显示<br>
speed_widget 小车移动速度<br>
turn_gain_widget 小车转动速度<br>
<span style="color: green;">
在Jupyter Notebook或JupyterLab中，display函数和ipywidgets库一起使用，可以创建和显示交互式小部件（widgets）。display(widgets.VBox([...]))调用展示了如何使用VBox（垂直框）和HBox（水平框）来组织小部件的布局。<br>
display函数：这个函数是Jupyter Notebook和JupyterLab特有的，用于在输出单元中显示对象。在这里，它被用来显示一个VBox小部件，该小部件包含了其他小部件。<br>
widgets.VBox：VBox是ipywidgets库中的一个类，用于创建一个垂直布局的容器。它接受一个列表作为参数，该列表包含要垂直排列的小部件。<br>
widgets.HBox([image_widget])：这里创建了一个HBox（水平框），它本身是一个容器，用于将小部件水平排列。然而，在这个例子中，HBox只包含了一个小部件——image_widget。<br>
label_widget、speed_widget、turn_gain_widget：这些是已经创建的小部件，分别用于显示标签、控制速度和转向增益。它们被直接添加到VBox中，因此会垂直排列。<br>
布局：最终，这个VBox容器会按照以下顺序垂直排列小部件：首先是一个包含image_widget的HBox（尽管在这个特定情况下，HBox只包含了一个元素），然后是label_widget、speed_widget和turn_gain_widget<br>。
当display函数被调用时，它会在Jupyter Notebook或JupyterLab的输出单元中渲染这个VBox容器，用户就可以看到这个垂直排列的小部件布局了。
<span>


In [12]:
import numpy as np

def rgb_to_jpeg(rgb_image):

    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY),90] #设置JPEG的质量
    result,encoded_img = cv2.imencode('.jpg',rgb_image,encode_param)
    if result:
        return encoded_img.tobytes()
    else:
        raise ValueError("Image encoding failed")
        
        
def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection


定义rgb_to_jpeg、detection_center、norm、closest_detection函数。用于改变图像颜色、检测目标物中心等。<br>
<span style="color: green;">
1.定义rgb_to_jpeg函数，该函数接受一个RGB图像（NumPy数组）作为输入，并返回其JPEG编码的字节流。<br>
设置JPEG质量：
cv2.IMWRITE_JPEG_QUALITY是一个OpenCV定义的标志，用于指定在保存JPEG图像时使用的质量。90是质量值，范围从0（最差质量，最小文件大小）到100。
编码图像：<br>
使用cv2.imencode函数将图像编码为JPEG格式。这个函数需要图像数据的格式是OpenCV期望的格式，即BGR顺序而不是RGB。因此，如果您的输入图像是RGB格式的，您需要先将其转换为BGR。<br>
处理结果：<br>
cv2.imencode返回一个布尔值和一个编码后的图像（如果成功的话）。布尔值表示操作是否成功，而第二个返回值是编码后的图像数据（如果操作成功）。<br>
2.detection_center(detection)<br>
这个函数接受一个检测对象（detection）作为输入，该对象应该包含一个键'bbox'，其值是一个包含四个元素的列表或元组，表示边界框的坐标。这四个元素通常定义为(x_min, y_min, width, height)，即边界框左上角的x和y坐标，以及边界框的宽度和高度。<br>
函数的目标是计算这个边界框中心的x和y坐标。不过，这里有一个需要注意的点：计算中心坐标后，它从每个坐标中减去了0.5。这可能是为了将坐标系统调整为某种特定的、可能是以图像中心为原点的系统，但通常这不是计算边界框中心的标准方式。标准方式只涉及将(x_min + width / 2, y_min + height / 2)作为中心坐标。<br>
3.norm(vec)<br>
这个函数接受一个二维向量（vec）作为输入，并返回该向量的长度（或称为欧几里得距离、模或范数）。这是通过计算向量各分量平方和的平方根来实现的，即np.sqrt(vec[0]**2 + vec[1]**2)。这个函数用于后续计算检测对象中心到图像中心（或假定原点）的距离。<br>
4.closest_detection(detections)<br>
这个函数接受一个检测对象列表（detections）作为输入，目标是找出这个列表中哪个检测对象的中心最接近图像的中心（或假定原点，如果之前detection_center函数中的坐标调整是为了这个目的的话）。<br>
函数遍历每个检测对象，计算其中心的坐标，并计算这个中心到原点（或图像中心）的距离。通过比较这些距离，函数最终返回中心最接近原点的检测对象。<br>
注意：如果detections列表为空，或者如果所有检测对象的中心都在一个与原点等距的圆上，这个函数将返回列表中的第一个检测对象作为“最接近”的，因为它没有处理这种情况下的优先级冲突。<br>
此外，由于detection_center函数中的坐标调整（减去0.5），这个“最接近”的计算实际上是基于一个特定的、可能是非标准的坐标系统。如果这不是预期的行为，可能需要调整detection_center函数，以返回标准的边界框中心坐标。<br>
<span>

In [13]:


from PIL import Image,ImageDraw,ImageFont
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
zh_font = ImageFont.truetype(font_path,13)        
def execute(change):
    labels=[]
    image = change['new']
   
    # compute all detected objects
    detections = model(image)

    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        labelv = det['label']
        text = data[labelv]['display_name']
        #labels.append(det['id'])
        #print(data[labelv][key])
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), 
                                                                      int(height * bbox[3])), (0, 0, 0), 1)
        cv2.putText(image,text,(int(width * bbox[0]),int(height * bbox[1])),font,font_scale,text_color,thickness)
        '''
        frame_pil = Image.fromarray(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(frame_pil)
        draw.text((int(width * bbox[0]),int(height * bbox[1])),text,font=zh_font,fill=(255,255,255,1))
        frame_with_text = cv2.cvtColor(np.array(frame_pil),cv2.COLOR_RGB2BGR)
        '''
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    
   
    if det is not None:
        bbox = det['bbox']
        #cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), 
        #int(height * bbox[3])), (0,0,0), 1)
        
    rgb_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image_widget.value = rgb_to_jpeg(image)
    out.write(rgb_image)
    
execute({'new': camera.value})


这段代码是一个处理图像检测结果的函数，它使用了OpenCV（通过cv2）和PIL（Python Imaging Library，通过PIL.Image等模块）来在图像上绘制检测到的对象的边界框和标签，并处理特定的检测结果。<br>
<span style="color: green;">
函数execute(change)
目的：接收一个新图像change，处理图像，并在图像上绘制检测到的对象的边界框和标签。<br>
参数：change是图像，其中'new'键对应的值是一个图像（NumPy数组或其他可转换为图像的格式）。<br>
流程<br>
初始化：创建一个空列表labels（尽管在后续代码中未使用），并获取change字典中'new'键对应的图像。<br>
检测对象：调用一个名为model的函数（前面代码中定义）来处理图像，并返回检测结果。model就是'ssd_mobilenet_v2_coco.engine'，是图像检测模型。<br>
绘制检测结果：<br>
遍历检测结果中的每个对象。<br>
使用边界框（bbox）和标签（label）来绘制矩形框和文本标签。<br>
这里使用了OpenCV的cv2.rectangle和cv2.putText函数。注意，width和height是图像的宽度和高度。<br>
筛选特定类别的检测结果：<br>
根据label_widget.value（未在代码中定义，可能是一个UI组件的值）来筛选匹配特定类别的检测结果。<br>
找到最接近中心的检测结果（假设closest_detection是一个未定义的函数）：<br>
在筛选后的检测结果中找到最接近图像中心的检测结果。<br>
处理特定检测结果（如果找到）：<br>
绘制该检测结果的边界框（但注释掉了绘制代码）。<br>
转换和显示图像：<br>
将图像从BGR格式转换为RGB格式。<br>
image_widget.value是一个UI组件，用于显示处理后的图像。这里使用了rgb_to_jpeg函数来将图像转换为JPEG格式的字节流。<br>
out是一个视频文件写入器或类似的输出流，用于保存处理后的图像。<br>
<span>


In [14]:
robot.left(speed=0.075)
robot.right(speed=-0.075)

设定小车转动速度并开始转动。

In [15]:
camera.unobserve_all()
camera.observe(execute, names='value')#这里开始循环

启动摄像头，开始进行视觉识别。<br>
<span style="color: green;">
camera.unobserve_all()
移除之前绑定到camera对象上的所有回调（callbacks）函数。在Widgets库中，当你想要在某个Widget的值发生变化时执行某些操作，你会使用.observe()方法将一个回调函数（比如execute函数）绑定到该Widget上。如果之后你不再需要这些回调函数，或者想要重新绑定新的回调函数，你可以使用.unobserve_all()来移除所有现有的回调函数，以避免不必要的执行或潜在的错误。<br>
camera.observe(execute, names='value')
将execute函数绑定为camera对象的观察者，以便在camera对象的value属性发生变化时自动调用execute函数。camera是上述捕获图像Widget，而value属性则是当前捕获到的图像或视频帧。<br>
<span>


In [16]:
import time
out.release()
camera.unobserve_all()
time.sleep(1.0)
robot.stop()

out.release()释放视频输出，关闭视频文件。<br>
停止摄像头工作和机器人运动。<br>
<span style="color: green;">
camera.unobserve_all():移除了绑定到camera对象上的所有观察者（回调函数）。<br>
在Widgets库中，.unobserve_all()方法用于确保当camera的属性（如捕获的图像或视频帧）更新时，不再调用任何先前绑定的回调函数。这通常用于清理资源，避免在对象不再需要时执行不必要的操作。<br>
time.sleep(1.0):<br>
time.sleep()函数接受一个浮点数作为参数，表示程序应该暂停的秒数。暂停可能是为了确保前面的操作（如释放资源和移除观察者）有足够的时间完成，或者是在停止机器人之前给系统一些时间来处理任何挂起的操作。<br>
robot.stop():<br>
调用了robot对象的stop()方法。robot很可能是控制物理机器人、机械臂或其他自动化设备的对象。stop()方法的具体行为将取决于robot对象的实现，但通常它会停止机器人的所有运动.<br>
<br>